In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from data_process import TextEyeTrackingDataset
import numpy as np

# torch.manual_seed(seed)
# np.random.seed(seed)

# 加载数据
dataset = TextEyeTrackingDataset('data/not_skip_data.csv')
batch_size = 1

# 假设dataset是你已经加载的完整数据集
train_size = int(len(dataset) * 0.8)  # 以80%的数据作为训练集
test_size = len(dataset) - train_size  # 剩余20%的数据作为测试集
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch.nn as nn
from model import MultiModalModelWithAttention
from train_and_evaluate import train_model
import warnings
from sklearn import exceptions

# 忽略特定类型的警告
warnings.filterwarnings("ignore", category=exceptions.UndefinedMetricWarning)

hidden_dim = 128  # LSTM隐藏层维度
output_dim = 1  # 输出维度，二分类问题

model = MultiModalModelWithAttention(hidden_dim, output_dim, show_dimension=False).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

pos_weight = torch.tensor([40])  # 大于1的值，表示正类相对于负类的权重
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)

tensorboard_dir = 'tensorlog/after/not-skip-attention'
# 训练模型
train_model(model, train_loader, test_loader, optimizer, criterion, device, tensorboard_dir, epochs=100)

-----Epoch 1-------
Train metric in Epoch 1, Loss: 0.013223999179899693, Accuracy: 0.98, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
Test metric in Epoch 1, Loss: 0.06738384813070297, Accuracy: 0.96, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
-----Epoch 2-------
Train metric in Epoch 2, Loss: 0.013886561617255211, Accuracy: 0.98, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
Test metric in Epoch 2, Loss: 0.06738384813070297, Accuracy: 0.96, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
-----Epoch 3-------
Train metric in Epoch 3, Loss: 0.009549775160849094, Accuracy: 1.0, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
Test metric in Epoch 3, Loss: 0.06738384813070297, Accuracy: 0.96, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
-----Epoch 4-------
Train metric in Epoch 4, Loss: 0.015039666555821896, Accuracy: 0.97, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
Test metric in Epoch 4, Loss: 0.06738384813070297, Accuracy: 0.96, AUC: 0.5, Precision:

In [ ]:
!tensorboard --logdir='tensorlog/after/not-skip-attention'

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.16.2 at http://localhost:6007/ (Press CTRL+C to quit)
